# 🎯 Кластеризация каротажных данных для калибровки генератора

Этот notebook выполняет кластеризацию реальных каротажных данных для определения основных петрофизических типов пород и извлечения параметров для генератора синтетических скважин.

## 🔬 Методология:
1. Загрузка каротажных данных (DTP, Neutron, Resistivity, GR, Density)
2. Предобработка и нормализация данных
3. Кластеризация методами K-Means, DBSCAN, Gaussian Mixture
4. Анализ кластеров и определение оптимального количества
5. Извлечение статистических параметров ПО КЛАСТЕРАМ
6. Генерация конфигурации для синтетического генератора

## 📊 Требуемые данные:
CSV файл с колонками:
- `Depth` - глубина (м)
- `GR` - гамма-каротаж (API)
- `Density` или `Rho` - плотность (г/см³)
- `Neutron` или `NPHI` - нейтронный каротаж (доли)
- `DTP` или `DT` или `AC` - акустический каротаж (мкс/м)
- `Resistivity` или `RT` - сопротивление (Ом·м)
- `Lithology` - литология (опционально, для валидации)

## 1. Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import json
import warnings
warnings.filterwarnings('ignore')

# Кластеризация
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

# Настройка стиля
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 8)

print("✅ Библиотеки загружены!")

## 2. Загрузка и подготовка данных

In [ ]:
# ИЗМЕНИТЕ ПУТЬ К ВАШЕМУ ФАЙЛУ
input_file = 'real_well_logs.csv'

# Загрузка данных
try:
    df = pd.read_csv(input_file)
    print(f"✅ Данные загружены: {df.shape[0]} строк × {df.shape[1]} столбцов")
    print(f"\nДоступные колонки: {', '.join(df.columns)}")
    display(df.head())
except FileNotFoundError:
    print(f"❌ Файл {input_file} не найден!")
    print("\n💡 Создаем ДЕМО-данные для примера...")
    
    # Создание демо-данных
    np.random.seed(42)
    n = 800
    
    # 4 типа пород с разными свойствами
    cluster_types = []
    gr_vals, rho_vals, nphi_vals, dt_vals, rt_vals = [], [], [], [], []
    
    # Тип 1: Песок (высокое сопротивление, низкий GR)
    n1 = 200
    cluster_types.extend([0] * n1)
    gr_vals.extend(np.random.normal(45, 8, n1))
    rho_vals.extend(np.random.normal(2.30, 0.08, n1))
    nphi_vals.extend(np.random.normal(0.25, 0.04, n1))
    dt_vals.extend(np.random.normal(200, 15, n1))
    rt_vals.extend(np.random.lognormal(1.5, 0.5, n1))
    
    # Тип 2: Глина (высокий GR, низкое сопротивление)
    n2 = 300
    cluster_types.extend([1] * n2)
    gr_vals.extend(np.random.normal(95, 12, n2))
    rho_vals.extend(np.random.normal(2.45, 0.06, n2))
    nphi_vals.extend(np.random.normal(0.35, 0.05, n2))
    dt_vals.extend(np.random.normal(260, 20, n2))
    rt_vals.extend(np.random.lognormal(0.2, 0.3, n2))
    
    # Тип 3: Карбонат (средний GR, высокая плотность)
    n3 = 150
    cluster_types.extend([2] * n3)
    gr_vals.extend(np.random.normal(35, 6, n3))
    rho_vals.extend(np.random.normal(2.70, 0.05, n3))
    nphi_vals.extend(np.random.normal(0.12, 0.03, n3))
    dt_vals.extend(np.random.normal(180, 12, n3))
    rt_vals.extend(np.random.lognormal(1.2, 0.4, n3))
    
    # Тип 4: Алевролит (промежуточные свойства)
    n4 = 150
    cluster_types.extend([3] * n4)
    gr_vals.extend(np.random.normal(65, 10, n4))
    rho_vals.extend(np.random.normal(2.50, 0.07, n4))
    nphi_vals.extend(np.random.normal(0.20, 0.04, n4))
    dt_vals.extend(np.random.normal(230, 18, n4))
    rt_vals.extend(np.random.lognormal(0.8, 0.4, n4))
    
    # Перемешивание
    indices = np.random.permutation(n)
    
    df = pd.DataFrame({
        'Depth': np.linspace(1000, 1800, n),
        'GR': np.array(gr_vals)[indices],
        'Density': np.array(rho_vals)[indices],
        'Neutron': np.array(nphi_vals)[indices],
        'DTP': np.array(dt_vals)[indices],
        'Resistivity': np.array(rt_vals)[indices],
        'True_Cluster': np.array(cluster_types)[indices]
    })
    
    print("✅ ДЕМО-данные созданы (4 типа пород)")
    display(df.head())
    print("\n⚠️ Замените путь на ваш файл с реальными данными!")

## 3. Выбор и подготовка признаков для кластеризации

In [ ]:
# Определение доступных колонок
feature_mapping = {
    'GR': ['GR', 'Gamma', 'GAMMA'],
    'Density': ['Density', 'Rho', 'RHOB', 'DEN'],
    'Neutron': ['Neutron', 'NPHI', 'NEU'],
    'DTP': ['DTP', 'DT', 'AC', 'DTCO'],
    'Resistivity': ['Resistivity', 'RT', 'RES', 'ILD', 'RILD']
}

# Поиск колонок в данных
selected_features = {}
for standard_name, possible_names in feature_mapping.items():
    for name in possible_names:
        if name in df.columns:
            selected_features[standard_name] = name
            break

print("📊 Найденные признаки для кластеризации:")
for standard, actual in selected_features.items():
    print(f"  • {standard:15s} → {actual}")

if len(selected_features) < 3:
    print("\n⚠️ ВНИМАНИЕ: Найдено менее 3 признаков. Рекомендуется минимум 3-5 признаков.")

# Создание DataFrame с выбранными признаками
feature_cols = list(selected_features.values())
df_features = df[feature_cols].copy()

# Удаление пропусков
df_features_clean = df_features.dropna()
print(f"\n📈 Данные для кластеризации: {len(df_features_clean)} точек × {len(feature_cols)} признаков")
print(f"   Удалено строк с пропусками: {len(df_features) - len(df_features_clean)}")

## 4. Обработка сопротивления (логарифмирование)

In [ ]:
# Логарифмирование сопротивления
if 'Resistivity' in selected_features:
    res_col = selected_features['Resistivity']
    
    # Проверка на положительные значения
    if (df_features_clean[res_col] <= 0).any():
        print(f"⚠️ Обнаружены неположительные значения в {res_col}. Заменяем на минимальное положительное.")
        min_pos = df_features_clean[df_features_clean[res_col] > 0][res_col].min()
        df_features_clean.loc[df_features_clean[res_col] <= 0, res_col] = min_pos
    
    df_features_clean['log_Resistivity'] = np.log10(df_features_clean[res_col])
    
    # Заменяем колонку сопротивления на логарифмическую
    feature_cols_processed = [col if col != res_col else 'log_Resistivity' for col in feature_cols]
    df_features_clean = df_features_clean[feature_cols_processed]
    
    print(f"✅ Сопротивление преобразовано: log10({res_col})")
else:
    feature_cols_processed = feature_cols
    print("ℹ️ Колонка Resistivity не найдена")

print(f"\nФинальные признаки: {', '.join(feature_cols_processed)}")

## 5. Нормализация данных

In [ ]:
# Статистика до нормализации
print("📊 Статистика признаков ДО нормализации:\n")
display(df_features_clean.describe())

# Нормализация (StandardScaler)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(df_features_clean)
df_scaled = pd.DataFrame(features_scaled, columns=feature_cols_processed, index=df_features_clean.index)

print("\n✅ Данные нормализованы (StandardScaler)")
print("\n📊 Статистика признаков ПОСЛЕ нормализации:\n")
display(df_scaled.describe())

## 6. Определение оптимального количества кластеров

Используем несколько методов для определения оптимального количества кластеров:

In [ ]:
# Диапазон количества кластеров для проверки
k_range = range(2, 11)

inertias = []
silhouette_scores = []
davies_bouldin_scores = []
calinski_harabasz_scores = []

print("🔍 Тестирование различного количества кластеров...\n")

for k in k_range:
    # K-Means
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(features_scaled)
    
    # Метрики
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(features_scaled, labels))
    davies_bouldin_scores.append(davies_bouldin_score(features_scaled, labels))
    calinski_harabasz_scores.append(calinski_harabasz_score(features_scaled, labels))
    
    print(f"K={k}: Silhouette={silhouette_scores[-1]:.3f}, Davies-Bouldin={davies_bouldin_scores[-1]:.3f}")

print("\n✅ Анализ завершен!")

In [ ]:
# Визуализация метрик
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Elbow Method (Inertia)
axes[0, 0].plot(k_range, inertias, 'bo-', linewidth=2, markersize=8)
axes[0, 0].set_xlabel('Количество кластеров (K)')
axes[0, 0].set_ylabel('Inertia')
axes[0, 0].set_title('Elbow Method')
axes[0, 0].grid(alpha=0.3)

# 2. Silhouette Score (выше = лучше)
axes[0, 1].plot(k_range, silhouette_scores, 'go-', linewidth=2, markersize=8)
axes[0, 1].set_xlabel('Количество кластеров (K)')
axes[0, 1].set_ylabel('Silhouette Score')
axes[0, 1].set_title('Silhouette Score (выше = лучше)')
axes[0, 1].grid(alpha=0.3)
axes[0, 1].axhline(y=max(silhouette_scores), color='r', linestyle='--', alpha=0.5)

# 3. Davies-Bouldin Index (ниже = лучше)
axes[1, 0].plot(k_range, davies_bouldin_scores, 'ro-', linewidth=2, markersize=8)
axes[1, 0].set_xlabel('Количество кластеров (K)')
axes[1, 0].set_ylabel('Davies-Bouldin Index')
axes[1, 0].set_title('Davies-Bouldin Index (ниже = лучше)')
axes[1, 0].grid(alpha=0.3)
axes[1, 0].axhline(y=min(davies_bouldin_scores), color='r', linestyle='--', alpha=0.5)

# 4. Calinski-Harabasz Index (выше = лучше)
axes[1, 1].plot(k_range, calinski_harabasz_scores, 'mo-', linewidth=2, markersize=8)
axes[1, 1].set_xlabel('Количество кластеров (K)')
axes[1, 1].set_ylabel('Calinski-Harabasz Score')
axes[1, 1].set_title('Calinski-Harabasz Score (выше = лучше)')
axes[1, 1].grid(alpha=0.3)
axes[1, 1].axhline(y=max(calinski_harabasz_scores), color='r', linestyle='--', alpha=0.5)

plt.suptitle('Метрики качества кластеризации', fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

# Рекомендация
best_k_silhouette = k_range[np.argmax(silhouette_scores)]
best_k_db = k_range[np.argmin(davies_bouldin_scores)]

print(f"\n🎯 РЕКОМЕНДАЦИИ:")
print(f"  • По Silhouette Score: K = {best_k_silhouette}")
print(f"  • По Davies-Bouldin: K = {best_k_db}")
print(f"\n💡 Рекомендуется выбрать K между {best_k_db} и {best_k_silhouette}")

## 7. Кластеризация с выбранным количеством кластеров

In [ ]:
# ВЫБЕРИТЕ КОЛИЧЕСТВО КЛАСТЕРОВ
n_clusters = best_k_silhouette  # Можно изменить вручную

print(f"🎯 Выбрано количество кластеров: {n_clusters}\n")

# K-Means кластеризация
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
clusters = kmeans.fit_predict(features_scaled)

# Добавление кластеров к данным
df_features_clean['Cluster'] = clusters

# Статистика по кластерам
print("📊 Распределение точек по кластерам:\n")
cluster_counts = pd.Series(clusters).value_counts().sort_index()
for cluster_id, count in cluster_counts.items():
    percentage = (count / len(clusters)) * 100
    print(f"  Кластер {cluster_id}: {count:5d} точек ({percentage:5.2f}%)")

# Качество кластеризации
silhouette = silhouette_score(features_scaled, clusters)
davies_bouldin = davies_bouldin_score(features_scaled, clusters)

print(f"\n📈 Качество кластеризации:")
print(f"  • Silhouette Score: {silhouette:.3f}")
print(f"  • Davies-Bouldin Index: {davies_bouldin:.3f}")

## 8. Визуализация кластеров (PCA)

In [ ]:
# PCA для визуализации в 2D
pca = PCA(n_components=2)
features_pca = pca.fit_transform(features_scaled)

print(f"📊 PCA: объясненная дисперсия = {pca.explained_variance_ratio_.sum()*100:.1f}%")
print(f"  • PC1: {pca.explained_variance_ratio_[0]*100:.1f}%")
print(f"  • PC2: {pca.explained_variance_ratio_[1]*100:.1f}%")

# Визуализация
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 1. Кластеры в PCA пространстве
scatter = axes[0].scatter(features_pca[:, 0], features_pca[:, 1], 
                         c=clusters, cmap='tab10', alpha=0.6, s=30)
axes[0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)')
axes[0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)')
axes[0].set_title('Кластеры в PCA пространстве')
axes[0].grid(alpha=0.3)
plt.colorbar(scatter, ax=axes[0], label='Кластер')

# Центроиды
centroids_pca = pca.transform(kmeans.cluster_centers_)
axes[0].scatter(centroids_pca[:, 0], centroids_pca[:, 1], 
               c='red', marker='X', s=200, edgecolors='black', linewidths=2,
               label='Центроиды')
axes[0].legend()

# 2. Распределение кластеров
axes[1].bar(range(n_clusters), cluster_counts.values, color='steelblue', edgecolor='black')
axes[1].set_xlabel('Кластер')
axes[1].set_ylabel('Количество точек')
axes[1].set_title('Распределение по кластерам')
axes[1].grid(alpha=0.3, axis='y')

# Добавление процентов
for i, count in enumerate(cluster_counts.values):
    percentage = (count / len(clusters)) * 100
    axes[1].text(i, count + 10, f'{percentage:.1f}%', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

## 9. Анализ центроидов кластеров

In [ ]:
# Преобразование центроидов обратно в оригинальное пространство
centroids_original = scaler.inverse_transform(kmeans.cluster_centers_)
df_centroids = pd.DataFrame(centroids_original, columns=feature_cols_processed)

# Если было логарифмирование сопротивления, преобразуем обратно
if 'log_Resistivity' in df_centroids.columns and 'Resistivity' in selected_features:
    df_centroids['Resistivity'] = 10 ** df_centroids['log_Resistivity']
    df_centroids = df_centroids.drop('log_Resistivity', axis=1)

print("="*70)
print("🎯 ЦЕНТРОИДЫ КЛАСТЕРОВ (средние значения)")
print("="*70)
display(df_centroids.round(3))

# Характеристика кластеров
print("\n💡 Интерпретация кластеров:")
for i in range(n_clusters):
    print(f"\nКластер {i}:")
    if 'GR' in selected_features:
        gr_val = df_centroids.iloc[i][selected_features['GR'] if selected_features['GR'] in df_centroids.columns else feature_cols_processed[0]]
        if gr_val < 50:
            print(f"  • Низкий GR ({gr_val:.1f}) → возможно песок/карбонат")
        elif gr_val > 80:
            print(f"  • Высокий GR ({gr_val:.1f}) → возможно глина")
        else:
            print(f"  • Средний GR ({gr_val:.1f}) → возможно алевролит")

## 10. Извлечение статистики ПО КЛАСТЕРАМ

**Ключевой момент:** Статистику снимаем по кластерам, а не по литотипам!

In [ ]:
print("="*70)
print("📊 СТАТИСТИКА ПО КЛАСТЕРАМ")
print("="*70)

cluster_stats = {}

for cluster_id in range(n_clusters):
    cluster_data = df_features_clean[df_features_clean['Cluster'] == cluster_id]
    
    print(f"\n{'='*70}")
    print(f"Кластер {cluster_id} ({len(cluster_data)} точек, {len(cluster_data)/len(df_features_clean)*100:.1f}%)")
    print(f"{'='*70}")
    
    stats = {}
    
    for col in feature_cols_processed:
        if col != 'log_Resistivity':
            mean_val = cluster_data[col].mean()
            std_val = cluster_data[col].std()
            min_val = cluster_data[col].min()
            max_val = cluster_data[col].max()
            
            stats[col] = {
                'mean': float(mean_val),
                'std': float(std_val),
                'min': float(min_val),
                'max': float(max_val)
            }
            
            print(f"  {col:15s}: {mean_val:8.3f} ± {std_val:6.3f}  [{min_val:8.3f}, {max_val:8.3f}]")
    
    # Если было логарифмирование сопротивления
    if 'log_Resistivity' in cluster_data.columns and 'Resistivity' in selected_features:
        res_original = 10 ** cluster_data['log_Resistivity']
        stats['Resistivity'] = {
            'mean': float(res_original.mean()),
            'std': float(res_original.std()),
            'min': float(res_original.min()),
            'max': float(res_original.max())
        }
        print(f"  {'Resistivity':15s}: {res_original.mean():8.3f} ± {res_original.std():6.3f}  (обратное преобразование)")
    
    cluster_stats[f'cluster_{cluster_id}'] = stats

print(f"\n{'='*70}")
print("✅ Статистика по всем кластерам извлечена!")
print(f"{'='*70}")

## 11. Анализ переходов между кластерами

In [ ]:
print("="*70)
print("🔄 МАТРИЦА ПЕРЕХОДОВ МЕЖДУ КЛАСТЕРАМИ")
print("="*70)

# Матрица переходов
clusters_seq = df_features_clean['Cluster'].tolist()
transition_counts = np.zeros((n_clusters, n_clusters))

for i in range(len(clusters_seq) - 1):
    from_cluster = clusters_seq[i]
    to_cluster = clusters_seq[i + 1]
    transition_counts[from_cluster, to_cluster] += 1

# Нормализация
row_sums = transition_counts.sum(axis=1, keepdims=True)
row_sums[row_sums == 0] = 1
transition_matrix = transition_counts / row_sums

print("\nВероятности переходов:")
cluster_names = [f'C{i}' for i in range(n_clusters)]
df_trans = pd.DataFrame(transition_matrix, index=cluster_names, columns=cluster_names)
print(df_trans.round(3))

# Визуализация
plt.figure(figsize=(10, 8))
sns.heatmap(transition_matrix, annot=True, fmt='.3f', cmap='YlOrRd',
            xticklabels=cluster_names, yticklabels=cluster_names,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Матрица переходов между кластерами', fontsize=14, fontweight='bold', pad=20)
plt.xlabel('К кластеру')
plt.ylabel('От кластера')
plt.tight_layout()
plt.show()

## 12. Анализ длин серий кластеров

In [ ]:
print("="*70)
print("📏 ДЛИНЫ СЕРИЙ КЛАСТЕРОВ")
print("="*70)

series_lengths = defaultdict(list)
current_cluster = clusters_seq[0]
current_length = 1

for cluster in clusters_seq[1:]:
    if cluster == current_cluster:
        current_length += 1
    else:
        series_lengths[current_cluster].append(current_length)
        current_cluster = cluster
        current_length = 1

series_lengths[current_cluster].append(current_length)

series_range = {}
print("\nКластер    Min    Max    Mean   Median  Кол-во серий")
print("-" * 70)

for cluster_id in range(n_clusters):
    if cluster_id in series_lengths and len(series_lengths[cluster_id]) > 0:
        lengths = series_lengths[cluster_id]
        min_len = int(min(lengths))
        max_len = int(max(lengths))
        mean_len = np.mean(lengths)
        median_len = np.median(lengths)
        
        series_range[f'cluster_{cluster_id}'] = (min_len, max_len)
        
        print(f"C{cluster_id}         {min_len:4d}   {max_len:4d}   {mean_len:6.1f}   {median_len:6.1f}   {len(lengths):4d}")

## 13. Визуализация кластеров по глубине (если доступна)

In [ ]:
if 'Depth' in df.columns:
    # Получение соответствующих глубин
    depths = df.loc[df_features_clean.index, 'Depth']
    
    fig, axes = plt.subplots(1, 3, figsize=(16, 10), sharey=True)
    
    # 1. Кластеры по глубине
    scatter = axes[0].scatter([0]*len(depths), depths, c=clusters, 
                             cmap='tab10', s=50, alpha=0.7)
    axes[0].set_title('Кластеры по глубине')
    axes[0].set_ylabel('Глубина (м)')
    axes[0].set_xlim(-1, 1)
    axes[0].set_xticks([])
    axes[0].invert_yaxis()
    plt.colorbar(scatter, ax=axes[0], label='Кластер')
    
    # 2-3. Каротажные кривые с раскраской по кластерам
    if 'GR' in selected_features:
        gr_col = selected_features['GR']
        gr_vals = df.loc[df_features_clean.index, gr_col]
        for cluster_id in range(n_clusters):
            mask = clusters == cluster_id
            axes[1].plot(gr_vals[mask], depths[mask], 'o', alpha=0.6, 
                        label=f'C{cluster_id}', markersize=3)
        axes[1].set_xlabel('GR (API)')
        axes[1].set_title('ГК по кластерам')
        axes[1].legend()
        axes[1].grid(alpha=0.3)
    
    if 'Density' in selected_features:
        rho_col = selected_features['Density']
        rho_vals = df.loc[df_features_clean.index, rho_col]
        for cluster_id in range(n_clusters):
            mask = clusters == cluster_id
            axes[2].plot(rho_vals[mask], depths[mask], 'o', alpha=0.6,
                        label=f'C{cluster_id}', markersize=3)
        axes[2].set_xlabel('Density (г/см³)')
        axes[2].set_title('Плотность по кластерам')
        axes[2].legend()
        axes[2].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 14. Сохранение конфигурации для генератора

In [ ]:
print("="*70)
print("💾 СОЗДАНИЕ КОНФИГУРАЦИИ ДЛЯ ГЕНЕРАТОРА")
print("="*70)

# Создание конфигурации
config = {
    'metadata': {
        'method': 'clustering',
        'algorithm': 'K-Means',
        'n_clusters': n_clusters,
        'n_points': len(df_features_clean),
        'features_used': feature_cols_processed,
        'silhouette_score': float(silhouette),
        'davies_bouldin_score': float(davies_bouldin)
    },
    'cluster_frequencies': {},
    'cluster_series_lengths': series_range,
    'cluster_properties': cluster_stats,
    'transition_matrix': transition_matrix.tolist(),
    'cluster_states': [f'cluster_{i}' for i in range(n_clusters)]
}

# Частоты кластеров
for cluster_id in range(n_clusters):
    freq = cluster_counts[cluster_id] / len(clusters)
    config['cluster_frequencies'][f'cluster_{cluster_id}'] = (
        max(0.01, freq * 0.8), 
        min(1.0, freq * 1.2)
    )

# Сохранение
output_file = 'clustered_config.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(config, f, indent=2, ensure_ascii=False)

print(f"\n✅ Конфигурация сохранена: {output_file}")
print(f"📊 Размер: {len(json.dumps(config))} байт")

print("\n📋 Содержимое:")
for key in config.keys():
    print(f"  • {key}")

print("\n🎯 Готово! Используйте конфигурацию для генерации синтетических скважин.")

## 15. Просмотр финальной конфигурации

In [ ]:
print("📄 ФИНАЛЬНАЯ КОНФИГУРАЦИЯ:\n")
print(json.dumps(config, indent=2, ensure_ascii=False))

## 📝 Заключение

### ✅ Что сделано:
1. Загружены и подготовлены каротажные данные
2. Выполнена кластеризация по 5 параметрам (DTP, Neutron, log(Resistivity), GR, Density)
3. Определено оптимальное количество кластеров
4. Извлечена **статистика по кластерам** (не по литотипам!)
5. Построена матрица переходов между кластерами
6. Проанализированы длины серий
7. Создан конфигурационный файл для генератора

### 🎯 Преимущества кластерного подхода:
- **Объективность**: кластеры определяются данными, а не экспертной разметкой
- **Петрофизическая основа**: группировка по реальным физическим свойствам
- **Воспроизводимость**: не зависит от субъективной интерпретации литологии
- **Детальность**: может выявить подтипы, которые не видны в литологической разметке

### 📊 Использование конфигурации:
```python
import json
with open('clustered_config.json', 'r') as f:
    config = json.load(f)

# Параметры кластеров
cluster_properties = config['cluster_properties']
transition_matrix = np.array(config['transition_matrix'])
```

---
🎯 **Кластеризация каротажных данных** | Генератор синтетических скважин | 2025